# TP3 : Court-circuit triphasé d'un alternateur
## Partie Manipulation

**Auteurs:** Wadii Sougrati & Mohammed Amine Yousfi  
**Date:** Mars 2025

Ce notebook implémente la partie manipulation du TP2 sur le court-circuit triphasé d'un alternateur. L'objectif est de déterminer l'évolution temporelle des grandeurs d'un alternateur à rotor bobiné avec amortisseurs lors d'un court-circuit triphasé au stator.

In [1]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from scipy import signal

# Définition des symboles pour le calcul symbolique
Rs, Ls, Rr, Lpr, Ra, Lpa, Mra, Msr, Msa, p = sp.symbols('R_s L_s R_r L_{pr} R_a L_{pa} M_{ra} M_{sr} M_{sa} p')

# Valeurs numériques
# Valeurs nominales
Vsn = 230      # Tension nominale [V]
fsn = 50       # Fréquence nominale [Hz]
Isn = 15.0     # Courant nominal [A]
Nn = 1500      # Vitesse nominale [tr/min]
cos_phi = -0.8  # Facteur de puissance (convention générateur)

# Valeurs des résistances et inductances
Rs_val = 2.90e-1   # Résistance statorique [Ω]
Ls_val = 4.96e-2   # Inductance cyclique statorique [H]
Rr_val = 2.00      # Résistance rotorique [Ω]
Lpr_val = 1.00     # Inductance propre rotorique [H]
Ra_val = 1.80      # Résistance amortisseurs [Ω]
Lpa_val = 2.00e-2  # Inductance propre amortisseurs [H]
Mra_val = 1.40e-1  # Inductance mutuelle rotor/amortisseur d'axe d [H]
Msr_val = 1.75e-1  # Inductance mutuelle stator/rotor [H]
Msa_val = 2.50e-2  # Inductance mutuelle stator/amortisseurs [H]

# Nombre de paires de pôles (pour une machine à 1500 tr/min à 50 Hz)
p_poles = 2

# Vitesse angulaire nominale
Omega0 = 2*np.pi*fsn  # [rad/s]

# Coefficient de frottement (pour le calcul du couple mécanique)
f_coeff = 0.1

1. Détermination du point de repos,

In [9]:
# Définition de la matrice des résistances [R]
R = sp.Matrix([
    [Rs, 0, 0, 0, 0],
    [0, Rs, 0, 0, 0],
    [0, 0, Rr, 0, 0],
    [0, 0, 0, Ra, 0],
    [0, 0, 0, 0, Ra]
])

# Définition de la matrice des inductances [L]
L = sp.Matrix([
    [Ls, 0, Msr, Msa, 0],
    [0, Ls, 0, 0, Msa],
    [3/2*Msr, 0, Lpr, Mra, 0],
    [3/2*Msa, 0, Mra, Lpa, 0],
    [0, 3/2*Msa, 0, 0, Lpa]
])

# Définition de la matrice des pulsations [omega0]
omega0 = sp.Matrix([
    [0, -Omega0, 0, 0, 0],
    [Omega0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0]
])

# Calcul de la matrice M(p) = R + omega0*L + p*L
M_p = R + omega0 * L + p * L

# Substitution des valeurs numériques
M_p_num = M_p.subs({
    Rs: Rs_val, 
    Ls: Ls_val, 
    Rr: Rr_val, 
    Lpr: Lpr_val, 
    Ra: Ra_val, 
    Lpa: Lpa_val, 
    Mra: Mra_val, 
    Msr: Msr_val, 
    Msa: Msa_val
})

print("Matrice M(p) définie symboliquement")
print("Dimensions: ", M_p.shape)
display(M_p)

Matrice M(p) définie symboliquement
Dimensions:  (5, 5)


Matrix([
[         L_s*p + R_s, -314.159265358979*L_s,                M_{sr}*p,                M_{sa}*p, -314.159265358979*M_{sa}],
[314.159265358979*L_s,           L_s*p + R_s, 314.159265358979*M_{sr}, 314.159265358979*M_{sa},                 M_{sa}*p],
[        1.5*M_{sr}*p,                     0,          L_{pr}*p + R_r,                M_{ra}*p,                        0],
[        1.5*M_{sa}*p,                     0,                M_{ra}*p,          L_{pa}*p + R_a,                        0],
[                   0,          1.5*M_{sa}*p,                       0,                       0,           L_{pa}*p + R_a]])

In [3]:
M_p_num

Matrix([
[ 0.0496*p + 0.29, -31.1645991236107,          0.175*p,         0.025*p, -15.707963267949],
[31.1645991236107,   0.0496*p + 0.29, 109.955742875643, 15.707963267949,          0.025*p],
[        0.2625*p,                 0,      1.0*p + 2.0,          0.14*p,                0],
[        0.0375*p,                 0,           0.14*p,    0.02*p + 1.8,                0],
[               0,          0.0375*p,                0,               0,     0.02*p + 1.8]])

2. Détermination de la matrice [M (p)] des variations,

In [11]:
phi = np.arccos(cos_phi)  # [rad]

# Calcul des courants statoriques en régime permanent
Isd0 = Isn * np.sin(phi)
Isq0 = Isn * np.cos(phi)

# Calcul des tensions statoriques en régime permanent
Vsd0 = Rs_val * Isd0 - Omega0 * Ls_val * Isq0
Vsq0 = Rs_val * Isq0 + Omega0 * Ls_val * Isd0

# Amplitude de la tension statorique
Vs = np.sqrt(Vsd0**2 + Vsq0**2)

# Calcul du courant rotorique en régime permanent
# En régime permanent, le courant dans les amortisseurs est nul
Id0 = 0
Iq0 = 0

# Pour le courant rotorique, on utilise les équations du régime permanent
# Vr0 = Rr*Ir0 (tension constante)
# Flux rotorique = Msr*Isd0 + Lr*Ir0 + Mra*Id0
# On simplifie en supposant que Ir0 est faible
Ir0 = -0.05  # Approximation

# Calcul des flux en régime permanent
Phisd0 = Ls_val * Isd0 + Msr_val * Ir0 + Msa_val * Id0
Phisq0 = Ls_val * Isq0 + Msa_val * Iq0
Phir0 = Msr_val * Isd0 + Lpr_val * Ir0 + Mra_val * Id0
Phid0 = Msa_val * Isd0 + Mra_val * Ir0 + Lpa_val * Id0
Phiq0 = Msa_val * Isq0 + Lpa_val * Iq0

# Calcul du couple électromagnétique en régime permanent
Ce0 = 1.5 * p_poles * ((Phisd0 * Isq0) - (Phisq0 * Isd0))

# Calcul du couple mécanique en régime permanent
Cm0 = Ce0 + f_coeff * Omega0

# Affichage des résultats
print(f"Courants statoriques: Isd0 = {Isd0:.2f} A, Isq0 = {Isq0:.2f} A")
print(f"Tensions statoriques: Vsd0 = {Vsd0:.2f} V, Vsq0 = {Vsq0:.2f} V")
print(f"Amplitude de la tension statorique: Vs = {Vs:.2f} V")
print(f"Courant rotorique: Ir0 = {Ir0:.2f} A")
print(f"Flux: Phisd0 = {Phisd0:.4f} Wb, Phisq0 = {Phisq0:.4f} Wb")
print(f"      Phir0 = {Phir0:.4f} Wb, Phid0 = {Phid0:.4f} Wb, Phiq0 = {Phiq0:.4f} Wb")
print(f"Couple électromagnétique: Ce0 = {Ce0:.2f} N.m")
print(f"Couple mécanique: Cm0 = {Cm0:.2f} N.m\n")

Courants statoriques: Isd0 = 9.00 A, Isq0 = -12.00 A
Tensions statoriques: Vsd0 = 376.59 V, Vsq0 = 277.00 V
Amplitude de la tension statorique: Vs = 467.49 V
Courant rotorique: Ir0 = -0.05 A
Flux: Phisd0 = 0.4376 Wb, Phisq0 = -0.5952 Wb
      Phir0 = 1.5250 Wb, Phid0 = 0.2180 Wb, Phiq0 = -0.3000 Wb
Couple électromagnétique: Ce0 = 0.32 N.m
Couple mécanique: Cm0 = 31.73 N.m



In [5]:
phi*180/3.14159

143.13022325108355

3. Détermination des fonctions de transferts Hi j (p) par inversion de [M (p)] (5),

In [6]:
detM = M_p.det(method = 'berkowitz').expand()
print(f"Déterminant de M(p) pour p=0: ")
display(detM)

# Calcul de la matrice des cofacteurs pour p=0
 
# Calcul de l'inverse de M(p)
M_inv_p0=np.zeros_like(M_p)
for i in range(5):
    for j in range(5):
        M_inv_p0[i,j] = M_p.cofactor(i, j, method='berkowitz').expand()/detM 

# Extraction des fonctions de transfert
H_Sdd_p0 = M_inv_p0[0, 0]
H_Sdq_p0 = M_inv_p0[0, 1]
H_Sqd_p0 = M_inv_p0[1, 0]
H_Sqq_p0 = M_inv_p0[1, 1]
H_Rd_p0 = M_inv_p0[2, 0]
H_Rq_p0 = M_inv_p0[2, 1]
H_Add_p0 = M_inv_p0[3, 0]
H_Adq_p0 = M_inv_p0[3, 1]
H_Aqd_p0 = M_inv_p0[4, 0]
H_Aqq_p0 = M_inv_p0[4, 1]

print("Fonctions de transfert:")

display(H_Sdd_p0)
display(H_Sdq_p0)
display(H_Sqd_p0)
display(H_Sqq_p0)
display(H_Rd_p0)
display(H_Rq_p0)
display(H_Add_p0)
display(H_Adq_p0)
display(H_Aqq_p0)






Déterminant de M(p) pour p=0: 


2.3283064365387e-10*L_s**4*R_s*p**2 - 1.16415321826935e-10*L_s**3*M_{sa}**2*p**3 + 2.3283064365387e-10*L_s**3*R_s**2*p + L_s**2*L_{pa}**2*L_{pr}*p**5 + 394784.176043574*L_s**2*L_{pa}**2*L_{pr}*p**3 + L_s**2*L_{pa}**2*R_r*p**4 + 394784.176043574*L_s**2*L_{pa}**2*R_r*p**2 + 2*L_s**2*L_{pa}*L_{pr}*R_a*p**4 + 789568.352087149*L_s**2*L_{pa}*L_{pr}*R_a*p**2 - L_s**2*L_{pa}*M_{ra}**2*p**5 - 394784.176043574*L_s**2*L_{pa}*M_{ra}**2*p**3 - 1.16415321826935e-10*L_s**2*L_{pa}*M_{sa}**2*p**3 + 2*L_s**2*L_{pa}*R_a*R_r*p**3 + 789568.352087149*L_s**2*L_{pa}*R_a*R_r*p - 2.3283064365387e-10*L_s**2*L_{pr}*M_{sa}**2*p**3 + L_s**2*L_{pr}*R_a**2*p**3 + 394784.176043574*L_s**2*L_{pr}*R_a**2*p - L_s**2*M_{ra}**2*R_a*p**4 - 394784.176043574*L_s**2*M_{ra}**2*R_a*p**2 - 1.16415321826935e-10*L_s**2*M_{sa}**2*R_a*p**2 - 2.3283064365387e-10*L_s**2*M_{sa}**2*R_r*p**2 - 2.3283064365387e-10*L_s**2*M_{sa}**2*R_s*p**2 + L_s**2*R_a**2*R_r*p**2 + 394784.176043574*L_s**2*R_a**2*R_r + 2*L_s*L_{pa}**2*L_{pr}*R_s*p**4 - 1.5*

Fonctions de transfert:


(L_s*L_{pa}**2*L_{pr}*p**4 + L_s*L_{pa}**2*R_r*p**3 + 2*L_s*L_{pa}*L_{pr}*R_a*p**3 - L_s*L_{pa}*M_{ra}**2*p**4 + 2*L_s*L_{pa}*R_a*R_r*p**2 + L_s*L_{pr}*R_a**2*p**2 - L_s*M_{ra}**2*R_a*p**3 + L_s*R_a**2*R_r*p + L_{pa}**2*L_{pr}*R_s*p**3 + L_{pa}**2*R_r*R_s*p**2 - 1.5*L_{pa}*L_{pr}*M_{sa}**2*p**4 + 2*L_{pa}*L_{pr}*R_a*R_s*p**2 - L_{pa}*M_{ra}**2*R_s*p**3 - 1.5*L_{pa}*M_{sa}**2*R_r*p**3 + 2*L_{pa}*R_a*R_r*R_s*p - 1.5*L_{pr}*M_{sa}**2*R_a*p**3 + L_{pr}*R_a**2*R_s*p + 1.5*M_{ra}**2*M_{sa}**2*p**4 - M_{ra}**2*R_a*R_s*p**2 - 1.5*M_{sa}**2*R_a*R_r*p**2 + R_a**2*R_r*R_s)/(2.3283064365387e-10*L_s**4*R_s*p**2 - 1.16415321826935e-10*L_s**3*M_{sa}**2*p**3 + 2.3283064365387e-10*L_s**3*R_s**2*p + L_s**2*L_{pa}**2*L_{pr}*p**5 + 394784.176043574*L_s**2*L_{pa}**2*L_{pr}*p**3 + L_s**2*L_{pa}**2*R_r*p**4 + 394784.176043574*L_s**2*L_{pa}**2*R_r*p**2 + 2*L_s**2*L_{pa}*L_{pr}*R_a*p**4 + 789568.352087149*L_s**2*L_{pa}*L_{pr}*R_a*p**2 - L_s**2*L_{pa}*M_{ra}**2*p**5 - 394784.176043574*L_s**2*L_{pa}*M_{ra}**2*p*

(-628.318530717959*L_s*L_{pa}**2*L_{pr}*p**3 - 628.318530717959*L_s*L_{pa}**2*R_r*p**2 - 1256.63706143592*L_s*L_{pa}*L_{pr}*R_a*p**2 + 628.318530717959*L_s*L_{pa}*M_{ra}**2*p**3 - 1256.63706143592*L_s*L_{pa}*R_a*R_r*p - 628.318530717959*L_s*L_{pr}*R_a**2*p + 628.318530717959*L_s*M_{ra}**2*R_a*p**2 - 628.318530717959*L_s*R_a**2*R_r + 942.477796076938*L_{pa}**2*M_{sr}**2*p**3 + 942.477796076938*L_{pa}*L_{pr}*M_{sa}**2*p**3 - 1884.95559215388*L_{pa}*M_{ra}*M_{sa}*M_{sr}*p**3 + 942.477796076938*L_{pa}*M_{sa}**2*R_r*p**2 + 1884.95559215388*L_{pa}*M_{sr}**2*R_a*p**2 + 942.477796076938*L_{pr}*M_{sa}**2*R_a*p**2 - 1884.95559215388*M_{ra}*M_{sa}*M_{sr}*R_a*p**2 + 942.477796076938*M_{sa}**2*R_a*R_r*p + 942.477796076938*M_{sr}**2*R_a**2*p)/(2.3283064365387e-10*L_s**4*R_s*p**2 - 1.16415321826935e-10*L_s**3*M_{sa}**2*p**3 + 2.3283064365387e-10*L_s**3*R_s**2*p + L_s**2*L_{pa}**2*L_{pr}*p**5 + 394784.176043574*L_s**2*L_{pa}**2*L_{pr}*p**3 + L_s**2*L_{pa}**2*R_r*p**4 + 394784.176043574*L_s**2*L_{pa}**

...

(-L_s*L_{pa}**2*M_{sr}*p**4 - 394784.176043574*L_s*L_{pa}**2*M_{sr}*p**2 + L_s*L_{pa}*M_{ra}*M_{sa}*p**4 + 394784.176043574*L_s*L_{pa}*M_{ra}*M_{sa}*p**2 - 2*L_s*L_{pa}*M_{sr}*R_a*p**3 - 789568.352087149*L_s*L_{pa}*M_{sr}*R_a*p + L_s*M_{ra}*M_{sa}*R_a*p**3 + 394784.176043574*L_s*M_{ra}*M_{sa}*R_a*p - L_s*M_{sr}*R_a**2*p**2 - 394784.176043574*L_s*M_{sr}*R_a**2 - L_{pa}**2*M_{sr}*R_s*p**3 + L_{pa}*M_{ra}*M_{sa}*R_s*p**3 + 1.5*L_{pa}*M_{sa}**2*M_{sr}*p**4 + 592176.264065361*L_{pa}*M_{sa}**2*M_{sr}*p**2 - 2*L_{pa}*M_{sr}*R_a*R_s*p**2 - 1.5*M_{ra}*M_{sa}**3*p**4 - 592176.264065361*M_{ra}*M_{sa}**3*p**2 + M_{ra}*M_{sa}*R_a*R_s*p**2 + 1.5*M_{sa}**2*M_{sr}*R_a*p**3 + 592176.264065361*M_{sa}**2*M_{sr}*R_a*p - M_{sr}*R_a**2*R_s*p)/(2.3283064365387e-10*L_s**4*R_s*p**2 - 1.16415321826935e-10*L_s**3*M_{sa}**2*p**3 + 2.3283064365387e-10*L_s**3*R_s**2*p + L_s**2*L_{pa}**2*L_{pr}*p**5 + 394784.176043574*L_s**2*L_{pa}**2*L_{pr}*p**3 + L_s**2*L_{pa}**2*R_r*p**4 + 394784.176043574*L_s**2*L_{pa}**2*R_r*p*

(-628.318530717959*L_{pa}**2*M_{sr}*R_s*p**2 + 628.318530717959*L_{pa}*M_{ra}*M_{sa}*R_s*p**2 - 1256.63706143592*L_{pa}*M_{sr}*R_a*R_s*p + 628.318530717959*M_{ra}*M_{sa}*R_a*R_s*p - 628.318530717959*M_{sr}*R_a**2*R_s)/(2.3283064365387e-10*L_s**4*R_s*p**2 - 1.16415321826935e-10*L_s**3*M_{sa}**2*p**3 + 2.3283064365387e-10*L_s**3*R_s**2*p + L_s**2*L_{pa}**2*L_{pr}*p**5 + 394784.176043574*L_s**2*L_{pa}**2*L_{pr}*p**3 + L_s**2*L_{pa}**2*R_r*p**4 + 394784.176043574*L_s**2*L_{pa}**2*R_r*p**2 + 2*L_s**2*L_{pa}*L_{pr}*R_a*p**4 + 789568.352087149*L_s**2*L_{pa}*L_{pr}*R_a*p**2 - L_s**2*L_{pa}*M_{ra}**2*p**5 - 394784.176043574*L_s**2*L_{pa}*M_{ra}**2*p**3 - 1.16415321826935e-10*L_s**2*L_{pa}*M_{sa}**2*p**3 + 2*L_s**2*L_{pa}*R_a*R_r*p**3 + 789568.352087149*L_s**2*L_{pa}*R_a*R_r*p - 2.3283064365387e-10*L_s**2*L_{pr}*M_{sa}**2*p**3 + L_s**2*L_{pr}*R_a**2*p**3 + 394784.176043574*L_s**2*L_{pr}*R_a**2*p - L_s**2*M_{ra}**2*R_a*p**4 - 394784.176043574*L_s**2*M_{ra}**2*R_a*p**2 - 1.16415321826935e-10*L_s**

(-L_s*L_{pa}*L_{pr}*M_{sa}*p**4 - 394784.176043574*L_s*L_{pa}*L_{pr}*M_{sa}*p**2 + L_s*L_{pa}*M_{ra}*M_{sr}*p**4 + 394784.176043574*L_s*L_{pa}*M_{ra}*M_{sr}*p**2 - L_s*L_{pa}*M_{sa}*R_r*p**3 - 394784.176043574*L_s*L_{pa}*M_{sa}*R_r*p - L_s*L_{pr}*M_{sa}*R_a*p**3 - 394784.176043574*L_s*L_{pr}*M_{sa}*R_a*p + L_s*M_{ra}*M_{sr}*R_a*p**3 + 394784.176043574*L_s*M_{ra}*M_{sr}*R_a*p - L_s*M_{sa}*R_a*R_r*p**2 - 394784.176043574*L_s*M_{sa}*R_a*R_r - L_{pa}*L_{pr}*M_{sa}*R_s*p**3 + L_{pa}*M_{ra}*M_{sr}*R_s*p**3 - L_{pa}*M_{sa}*R_r*R_s*p**2 + 1.5*L_{pr}*M_{sa}**3*p**4 + 592176.264065361*L_{pr}*M_{sa}**3*p**2 - L_{pr}*M_{sa}*R_a*R_s*p**2 - 1.5*M_{ra}*M_{sa}**2*M_{sr}*p**4 - 592176.264065361*M_{ra}*M_{sa}**2*M_{sr}*p**2 + M_{ra}*M_{sr}*R_a*R_s*p**2 + 1.5*M_{sa}**3*R_r*p**3 + 592176.264065361*M_{sa}**3*R_r*p - M_{sa}*R_a*R_r*R_s*p)/(2.3283064365387e-10*L_s**4*R_s*p**2 - 1.16415321826935e-10*L_s**3*M_{sa}**2*p**3 + 2.3283064365387e-10*L_s**3*R_s**2*p + L_s**2*L_{pa}**2*L_{pr}*p**5 + 394784.176043574*L

(-628.318530717959*L_{pa}*L_{pr}*M_{sa}*R_s*p**2 + 628.318530717959*L_{pa}*M_{ra}*M_{sr}*R_s*p**2 - 628.318530717959*L_{pa}*M_{sa}*R_r*R_s*p - 628.318530717959*L_{pr}*M_{sa}*R_a*R_s*p + 628.318530717959*M_{ra}*M_{sr}*R_a*R_s*p - 628.318530717959*M_{sa}*R_a*R_r*R_s)/(2.3283064365387e-10*L_s**4*R_s*p**2 - 1.16415321826935e-10*L_s**3*M_{sa}**2*p**3 + 2.3283064365387e-10*L_s**3*R_s**2*p + L_s**2*L_{pa}**2*L_{pr}*p**5 + 394784.176043574*L_s**2*L_{pa}**2*L_{pr}*p**3 + L_s**2*L_{pa}**2*R_r*p**4 + 394784.176043574*L_s**2*L_{pa}**2*R_r*p**2 + 2*L_s**2*L_{pa}*L_{pr}*R_a*p**4 + 789568.352087149*L_s**2*L_{pa}*L_{pr}*R_a*p**2 - L_s**2*L_{pa}*M_{ra}**2*p**5 - 394784.176043574*L_s**2*L_{pa}*M_{ra}**2*p**3 - 1.16415321826935e-10*L_s**2*L_{pa}*M_{sa}**2*p**3 + 2*L_s**2*L_{pa}*R_a*R_r*p**3 + 789568.352087149*L_s**2*L_{pa}*R_a*R_r*p - 2.3283064365387e-10*L_s**2*L_{pr}*M_{sa}**2*p**3 + L_s**2*L_{pr}*R_a**2*p**3 + 394784.176043574*L_s**2*L_{pr}*R_a**2*p - L_s**2*M_{ra}**2*R_a*p**4 - 394784.176043574*L_s**2

(-L_s*L_{pa}*L_{pr}*M_{sa}*p**4 - 394784.176043574*L_s*L_{pa}*L_{pr}*M_{sa}*p**2 - L_s*L_{pa}*M_{sa}*R_r*p**3 - 394784.176043574*L_s*L_{pa}*M_{sa}*R_r*p - L_s*L_{pr}*M_{sa}*R_a*p**3 - 394784.176043574*L_s*L_{pr}*M_{sa}*R_a*p + L_s*M_{ra}**2*M_{sa}*p**4 + 394784.176043574*L_s*M_{ra}**2*M_{sa}*p**2 - L_s*M_{sa}*R_a*R_r*p**2 - 394784.176043574*L_s*M_{sa}*R_a*R_r - L_{pa}*L_{pr}*M_{sa}*R_s*p**3 + 1.5*L_{pa}*M_{sa}*M_{sr}**2*p**4 + 592176.264065361*L_{pa}*M_{sa}*M_{sr}**2*p**2 - L_{pa}*M_{sa}*R_r*R_s*p**2 + 1.5*L_{pr}*M_{sa}**3*p**4 + 592176.264065361*L_{pr}*M_{sa}**3*p**2 - L_{pr}*M_{sa}*R_a*R_s*p**2 + M_{ra}**2*M_{sa}*R_s*p**3 - 3.0*M_{ra}*M_{sa}**2*M_{sr}*p**4 - 1184352.52813072*M_{ra}*M_{sa}**2*M_{sr}*p**2 + 1.5*M_{sa}**3*R_r*p**3 + 592176.264065361*M_{sa}**3*R_r*p + 1.5*M_{sa}*M_{sr}**2*R_a*p**3 + 592176.264065361*M_{sa}*M_{sr}**2*R_a*p - M_{sa}*R_a*R_r*R_s*p)/(2.3283064365387e-10*L_s**4*R_s*p**2 - 1.16415321826935e-10*L_s**3*M_{sa}**2*p**3 + 2.3283064365387e-10*L_s**3*R_s**2*p + L_s**

In [7]:
detM

2.3283064365387e-10*L_s**4*R_s*p**2 - 1.16415321826935e-10*L_s**3*M_{sa}**2*p**3 + 2.3283064365387e-10*L_s**3*R_s**2*p + L_s**2*L_{pa}**2*L_{pr}*p**5 + 394784.176043574*L_s**2*L_{pa}**2*L_{pr}*p**3 + L_s**2*L_{pa}**2*R_r*p**4 + 394784.176043574*L_s**2*L_{pa}**2*R_r*p**2 + 2*L_s**2*L_{pa}*L_{pr}*R_a*p**4 + 789568.352087149*L_s**2*L_{pa}*L_{pr}*R_a*p**2 - L_s**2*L_{pa}*M_{ra}**2*p**5 - 394784.176043574*L_s**2*L_{pa}*M_{ra}**2*p**3 - 1.16415321826935e-10*L_s**2*L_{pa}*M_{sa}**2*p**3 + 2*L_s**2*L_{pa}*R_a*R_r*p**3 + 789568.352087149*L_s**2*L_{pa}*R_a*R_r*p - 2.3283064365387e-10*L_s**2*L_{pr}*M_{sa}**2*p**3 + L_s**2*L_{pr}*R_a**2*p**3 + 394784.176043574*L_s**2*L_{pr}*R_a**2*p - L_s**2*M_{ra}**2*R_a*p**4 - 394784.176043574*L_s**2*M_{ra}**2*R_a*p**2 - 1.16415321826935e-10*L_s**2*M_{sa}**2*R_a*p**2 - 2.3283064365387e-10*L_s**2*M_{sa}**2*R_r*p**2 - 2.3283064365387e-10*L_s**2*M_{sa}**2*R_s*p**2 + L_s**2*R_a**2*R_r*p**2 + 394784.176043574*L_s**2*R_a**2*R_r + 2*L_s*L_{pa}**2*L_{pr}*R_s*p**4 - 1.5*

3. Coefficients du polynôme dénominateur (du plus haut degré au plus bas):

In [8]:

# Extraction des coefficients du polynôme dénominateur
den_poly = detM
display(detM)

den_poly = den_poly.subs({
    Rs: Rs_val, 
    Ls: Ls_val, 
    Rr: Rr_val, 
    Lpr: Lpr_val, 
    Ra: Ra_val, 
    Lpa: Lpa_val, 
    Mra: Mra_val, 
    Msr: Msr_val, 
    Msa: Msa_val
})
print(den_poly)
poles=sp.solve(den_poly,p)



print("\nPôles du système:")
print(poles)

# Analyse des propriétés de la réponse indicielle
# Calcul des constantes de temps pour les pôles réels
time_constants = []
for pole in poles:
    if abs(pole.imag) < 1e-10:  # Pôle réel
        tau = -1/pole.real
        time_constants.append(tau)
        print(f"Constante de temps τ = {tau:.6f} s (pôle réel)")
    else:  # Pôle complexe
        zeta = -pole.real / np.sqrt(pole.real**2 + pole.imag**2)
        omega_n = np.sqrt(pole.real**2 + pole.imag**2)
        print(f"Amortissement ζ = {zeta:.6f}, Pulsation propre ωn = {omega_n:.6f} rad/s (pôle complexe)")

# Détermination du temps d'observation et de la période d'échantillonnage
if time_constants:
    tau_max = max(time_constants)
    tau_min = min(time_constants)
    T_OBS = 5 * tau_max
    T_e = tau_min / 20
else:
    # Si pas de pôles réels, utiliser les pôles complexes
    zeta_min = min([-pole.real / np.sqrt(pole.real**2 + pole.imag**2) for pole in poles if abs(pole.imag) > 1e-10])
    omega_n_min = min([np.sqrt(pole.real**2 + pole.imag**2) for pole in poles if abs(pole.imag) > 1e-10])
    T_OBS = 5 / (zeta_min * omega_n_min)
    T_e = 0.1 / omega_n_min

print(f"\nTemps d'observation recommandé: T_OBS = {T_OBS:.6f} s")
print(f"Période d'échantillonnage recommandée: T_e = {T_e:.6f} s\n")

2.3283064365387e-10*L_s**4*R_s*p**2 - 1.16415321826935e-10*L_s**3*M_{sa}**2*p**3 + 2.3283064365387e-10*L_s**3*R_s**2*p + L_s**2*L_{pa}**2*L_{pr}*p**5 + 394784.176043574*L_s**2*L_{pa}**2*L_{pr}*p**3 + L_s**2*L_{pa}**2*R_r*p**4 + 394784.176043574*L_s**2*L_{pa}**2*R_r*p**2 + 2*L_s**2*L_{pa}*L_{pr}*R_a*p**4 + 789568.352087149*L_s**2*L_{pa}*L_{pr}*R_a*p**2 - L_s**2*L_{pa}*M_{ra}**2*p**5 - 394784.176043574*L_s**2*L_{pa}*M_{ra}**2*p**3 - 1.16415321826935e-10*L_s**2*L_{pa}*M_{sa}**2*p**3 + 2*L_s**2*L_{pa}*R_a*R_r*p**3 + 789568.352087149*L_s**2*L_{pa}*R_a*R_r*p - 2.3283064365387e-10*L_s**2*L_{pr}*M_{sa}**2*p**3 + L_s**2*L_{pr}*R_a**2*p**3 + 394784.176043574*L_s**2*L_{pr}*R_a**2*p - L_s**2*M_{ra}**2*R_a*p**4 - 394784.176043574*L_s**2*M_{ra}**2*R_a*p**2 - 1.16415321826935e-10*L_s**2*M_{sa}**2*R_a*p**2 - 2.3283064365387e-10*L_s**2*M_{sa}**2*R_r*p**2 - 2.3283064365387e-10*L_s**2*M_{sa}**2*R_s*p**2 + L_s**2*R_a**2*R_r*p**2 + 394784.176043574*L_s**2*R_a**2*R_r + 2*L_s*L_{pa}**2*L_{pr}*R_s*p**4 - 1.5*

5.94050000000537e-11*p**5 + 4.7519094999997e-7*p**4 + 0.000711041753977127*p**3 + 0.253934022337216*p**2 + 240.516149930749*p + 6294.12987370913

Pôles du système:
[-6148.05812551402, -1726.80511854320, -26.8754872169205, -48.7177903223674 - 607.428533092957*I, -48.7177903223674 + 607.428533092957*I]


AttributeError: 'Float' object has no attribute 'imag'